In [1]:
import sys
import os
import warnings
import time

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
warnings.filterwarnings("ignore")

In [2]:
from src.database.db_connection import *
from src.database.preparation import *
from src.utilities.step1_simplify_fds import *
from src.utilities.step2_clustering_and_generating_repairs import *
from src.utilities.step3_convert_back_to_db import *
from src.utilities.step4_group_similar_sentences import *
from src.evaluation.metrics import *
from src.database.db_connection import *

In [3]:
# inconsistent_df = pd.DataFrame([
#     [101, 'John Bell', 'UT', 19],
#     [101, 'James Bell', 'UT', 20],
#     [102, 'Claire Stevens', 'TUE', 20],
#     [102, 'Claire Stevens', 'TUE', 20]
# ], columns=['SID','SName','University','Age'])

In [4]:
engine = create_engine_for_db()

In [5]:
with engine.begin() as connection:
    connection.execute(text("""
    TRUNCATE TABLE prototype_fp.students_prob
    RESTART IDENTITY
    CASCADE;
    """))
    connection.execute(text("""
    UPDATE prototype_fp."_dict"
    SET dict = dictionary('')
    WHERE name = :name;
    """), {"name": "students_dict"})

In [6]:
fd_constraints = [(['sid', 'university'],['sname', 'address', 'postcode', 'emailid', 'phonenum'])]

In [7]:
fd_constraints = simplify_FDs(fd_constraints)

In [8]:
fd_constraints

[(['sid', 'university'], ['sname']),
 (['sid', 'university'], ['address']),
 (['sid', 'university'], ['postcode']),
 (['sid', 'university'], ['emailid']),
 (['sid', 'university'], ['phonenum'])]

In [9]:
violation_levels = [{'target_violation_fraction':0.2, 'dups_per_cluster':(1,2), 'cells_corrupted':(1,2)},
{'target_violation_fraction':0.4, 'dups_per_cluster':(2,3), 'cells_corrupted':(2,3)},
{'target_violation_fraction':0.6, 'dups_per_cluster':(2,3), 'cells_corrupted':(3,4)},
{'target_violation_fraction':0.8, 'dups_per_cluster':(2,3), 'cells_corrupted':(3,4)}]

#violation_levels = [{'target_violation_fraction':0.2, 'dups_per_cluster':(1,2), 'cells_corrupted':(1,2)}]
# {'target_violation_fraction':0.5, 'dups_per_cluster':(2,3), 'cells_corrupted':(1,2)},
# {'target_violation_fraction':0.8, 'dups_per_cluster':(3,5), 'cells_corrupted':(2,3)},
# {'target_violation_fraction':1.0, 'dups_per_cluster':(6,10), 'cells_corrupted':(3,5)}]

num_inconsistent_relations = 20

delete_cost = 2
update_cost = 1
eps=1e-9
prob_type = 'UNIFORM'
constraint_hardness = [1.0] * len(fd_constraints)

query_file_name_expected_Prob = "../../../queries/FP/synthetic/Students_truth.sql"
query_file_name_actual_Prob = "../../../queries/FP/synthetic/Students_prob.sql"
algorithm_name = 'PRA'
old_table_name = ''
new_table_name = ''

j = 3
i = 20
table_name = f"students_{j}_{i}"

In [10]:
table_name = f"students_{j}_{i}"
inconsistent_df = pd.read_sql_table(table_name, con=engine, schema="prototype_fp")

In [11]:
start = time.perf_counter()
inconsistent_df = preprocess_data(inconsistent_df)
actions,cluster_ids,rv_probs = compute_actions(inconsistent_df, fd_constraints, constraint_hardness,delete_cost, update_cost,eps,prob_type)
all_actions,cartesian_possible,cartesian_computed = min_cost_actions_pruned(cluster_ids,actions,delete_cost,update_cost)

#repaired_versions =  apply_combined_actions(inconsistent_df, all_actions, uuid_col="uuid")
#repaired_versions = pd.concat(repaired_versions, ignore_index=True)
# repaired_versions = repaired_versions.drop(columns=["uuid"])
# rv_probabilities = filter_and_condition_min_cost_rvs(rv_probs, repaired_versions)
# repaired_df = group_similar_rows_together(repaired_versions)

In [36]:
actions

{1: {1: [{'FD': 1,
    'LHS': ['sid', 'university'],
    'Cluster': 1,
    'Tuples': {3, 12, 35, 40, 42},
    'TargetValue': 'David Wong',
    'DR': [24, 18],
    'UR': [],
    'Cost': 4,
    'min_cost': 2,
    'rv': 'fd1sname1=1',
    '_dr_set': frozenset({'18', '24'}),
    '_ur_tids': frozenset(),
    '_ur_cells': ()},
   {'FD': 1,
    'LHS': ['sid', 'university'],
    'Cluster': 1,
    'Tuples': {3, 12, 24, 35, 40, 42},
    'TargetValue': 'David Wong',
    'DR': [18],
    'UR': ['24:sname=David Wong'],
    'Cost': 3,
    'min_cost': 2,
    'rv': 'fd1sname1=2',
    '_dr_set': frozenset({'18'}),
    '_ur_tids': frozenset({'24'}),
    '_ur_cells': ((('24', 'sname'), 'David Wong'),)},
   {'FD': 1,
    'LHS': ['sid', 'university'],
    'Cluster': 1,
    'Tuples': {3, 12, 18, 35, 40, 42},
    'TargetValue': 'David Wong',
    'DR': [24],
    'UR': ['18:sname=David Wong'],
    'Cost': 3,
    'min_cost': 2,
    'rv': 'fd1sname1=3',
    '_dr_set': frozenset({'24'}),
    '_ur_tids': frozenset(

In [12]:
repaired_versions =  apply_combined_actions(inconsistent_df, all_actions, uuid_col="uuid")

In [13]:
repaired_versions = pd.concat(repaired_versions, ignore_index=True)
repaired_versions = repaired_versions.drop(columns=["uuid"])

In [14]:
#rv_probabilities = filter_and_condition_min_cost_rvs(rv_probs, repaired_versions)
rv_probabilities = rv_probs

In [15]:
repaired_df = group_similar_rows_together(repaired_versions)
repaired_df.to_sql('students_prob',engine,schema='prototype_fp',index=False,if_exists='append')

49

In [16]:
entries = [f"{k}:{v}" for k, v in rv_probabilities.items()]
joined = ';'.join(entries)

update_stmt = f"UPDATE prototype_fp._dict\nSET dict=add(dict, '{joined}')\nWHERE name='students_dict';"
with engine.begin() as connection:
    connection.execute(text(update_stmt))